In [121]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow import keras
%matplotlib inline
sns.set_style('whitegrid')

In [122]:
df_test_features = pd.read_csv('../test_features.csv')

In [123]:
num_test_data = df_test_features.iloc[:,9:]
# num_test_data['loudness'] = df_test_features['loudness']
# num_test_data['tempo'] = df_test_features['tempo']
# num_test_data['duration'] = df_test_features['duration']
cat_test_data = df_test_features[['time_signature', 'key', 'mode']]

In [124]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [125]:
cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
cat_test_data['mode_cat'] = le.fit_transform(cat_test_data['mode'])

C:\Users\alex0\AppData\Local\Temp/ipykernel_29404/719706249.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_29404/719706249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_29404/719706249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [126]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [127]:
num_test = scaler.fit_transform(num_test_data)

In [128]:
import ktrain
from ktrain import text

In [129]:
predictor = ktrain.load_predictor('../audio_meta_text_nbsvm_model')

In [130]:
time_signature_test = np.expand_dims(cat_test_data.time_signature_cat, -1)
key_test = np.expand_dims(cat_test_data.key_cat, -1)
mode_test = np.expand_dims(cat_test_data.mode_cat, -1)

# Pre-process string with ktrain

In [131]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [132]:
# remove stopwords
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [133]:
x_test2 = df_test_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
x_test2 = x_test2.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test2 = np.array(x_test2)

In [134]:
x_text_input = predictor.preproc.preprocess_test(x_test2)

428 test sequences
test sequence lengths:
	mean : 54
	95percentile : 109
	99percentile : 150
x_test shape: (428,150)


# Make a prediction

In [135]:
test_dataset = [num_test] + [x_text_input[0]] + [time_signature_test] + [key_test] + [mode_test]

In [136]:
prediction = predictor.model.predict(test_dataset)

In [137]:
predict_labels = [ label.argmax() for label in prediction]

In [138]:
df = pd.DataFrame(predict_labels, columns=['genre_pred'])

In [139]:
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}

In [140]:
df['genre_pred'] = df['genre_pred'].map(classes_mapping)

In [141]:
df['trackID'] = df_test_features['trackID']

In [142]:
correct_label = pd.read_csv('../test_labels.csv')

In [143]:
# merge df and correct_label by trackID
df_merged = pd.merge(df, correct_label, on='trackID')

In [144]:
ultimate_accuracy = len(df_merged[df_merged['genre_pred'] == df_merged['genre']])/len(df_merged)

In [145]:
ultimate_accuracy

0.4462616822429907

In [146]:
from sklearn.metrics import classification_report
print(classification_report(df_merged['genre'], df_merged['genre_pred']))

                       precision    recall  f1-score   support

 classic pop and rock       0.27      0.33      0.29        64
dance and electronica       0.46      0.19      0.27        64
                 folk       0.49      0.52      0.51        73
       jazz and blues       0.10      0.06      0.07        36
                metal       0.77      0.57      0.66        42
                  pop       0.68      0.82      0.74        44
                 punk       0.45      0.50      0.48        40
      soul and reggae       0.39      0.58      0.47        65

             accuracy                           0.45       428
            macro avg       0.45      0.45      0.44       428
         weighted avg       0.45      0.45      0.43       428

